In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-10-20 11:02 ajna         INFO     Configuração de log efetuada


2025-10-20 11:02:48,043 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [7]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

,codigoConteiner,codigoTerminalCarregamento,portoOrigemCarga,portoDestFinal,paisDestinoFinalMercante,portoCarregamento,portoDescarregamento
0,FCIU5666469,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
1,NYKU3617382,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
2,NYKU9779457,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
3,TCKU2348423,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
4,TCLU3061646,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
...,...,...,...,...,...,...,...
281889,HLBU1686944,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ
281890,HLBU1856336,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ
281891,HLBU1891511,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ
281892,TCLU9303270,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ


In [8]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [9]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id,
    c.placa,
    c.placaSemirreboque,
    c.numeroConteiner,
    c.vazioConteiner,
    c.codigoRecinto,
    c.cpfMotorista,
    c.nomeMotorista,
    c.dataHoraOcorrencia,
    c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner 
    AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (
        (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
        OR e.codigoRecinto = '8931404'
    )
    AND e.codigoRecinto != c.codigoRecinto
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)
# Atenção! A DPW/EMBRAPORT pode operar como REDEX e envia carga para a BTP como pode receber carga de um REDEX e operar como Terminal exportador
# Este código contempla estes dois casos da DPW/EMBRAPORT

,codigoTerminalCarregamento,portoDestFinal,paisDestinoFinalMercante,portoDescarregamento,conteineres
0,BRIGI001,LBBEY,LB,MAPTM,1
1,BRIOA001,IDJKT,ID,SGSIN,15
2,BRPNG002,CNSHA,CN,CNSHA,1
3,BRPNG002,GHTEM,GH,ZADUR,3
4,BRPNG002,ZADUR,ZA,ZADUR,1
...,...,...,...,...,...
2487,BRSSZ058,ZAPLZ,ZA,ESALG,99
2488,BRSSZ058,ZAPLZ,ZA,MAPTM,18
2489,BRSSZ058,ZAPLZ,ZA,NAWVB,30
2490,BRSSZ058,ZAZBA,ZA,ESLPA,133


In [11]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

codigoTerminalCarregamento
BRSSZ058    133776
BRSSZ016     95066
BRSSZ009     50200
BRSSZ057      2827
BRIOA001        15
BRPNG002         5
BRRIO002         2
BRSSZ031         2
BRIGI001         1
Name: conteineres, dtype: int64

In [13]:
df_entrada_operadores

codigoTerminalCarregamento
BRSSZ058    133776
BRSSZ016     95066
BRSSZ009     50200
BRSSZ057      2827
BRIOA001        15
BRPNG002         5
BRSSZ031         2
BRRIO002         2
BRIGI001         1
Name: count, dtype: int64

In [14]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

,paisDestinoFinalMercante,conteineres
27,CN,41264
149,US,25208
70,IN,12601
101,MX,11550
108,NL,9482
28,CO,8130
35,DE,6234
76,JP,6112
113,PE,6038
5,AR,6035


In [16]:
df_saida_redex

paisDestinoFinalMercante
CN    41264
US    25208
IN    12601
MX    11550
NL     9482
CO     8130
DE     6234
JP     6112
PE     6038
AR     6035
Name: count, dtype: int64

In [17]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

codigoTerminalCarregamento  paisDestinoFinalMercante
BRIGI001                    LB                              1
BRIOA001                    ID                             15
BRPNG002                    GH                              3
                            CN                              1
                            ZA                              1
BRRIO002                    CO                              1
                            US                              1
BRSSZ009                    US                          12064
                            CN                          11882
                            MX                           2912
                            CO                           2592
                            AR                           2237
                            KR                           1609
                            VN                           1601
                            ZA                           1497
                 

In [18]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


,sigla,nome,escaneamento
0,AE,Emirados Árabes Unidos,1
1,AG,Antígua e Barbuda,0
2,AL,Albânia,1
3,AN,Antilhas Holandesas,0
4,AO,Angola,1
...,...,...,...
143,VE,Venezuela,0
144,VI,Ilhas Virgens (U.S.),0
145,VN,Vietnã,0
146,YE,Iêmen,0


In [20]:
# df_saidas_entradas

In [56]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [57]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

In [58]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

105903 281894


In [59]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

In [60]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

,codigoTerminalCarregamento,conteineres
8,BRSSZ058,133776
5,BRSSZ016,95066
4,BRSSZ009,50200
7,BRSSZ057,2827
1,BRIOA001,15
2,BRPNG002,5
3,BRRIO002,2
6,BRSSZ031,2
0,BRIGI001,1


In [66]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

,codigoTerminalCarregamento,conteineres
5,BRSSZ058,86822
3,BRSSZ016,11612
2,BRSSZ009,4932
4,BRSSZ057,2532
1,BRPNG002,4
0,BRIGI001,1


In [25]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

,codigoTerminalDescarregamento,portoOrigemCarga,portoCarregamento,conteineres,paisOrigem
0,BRSSZ001,SIKOP,BRPNG,1,SI
1,BRSSZ009,AEDXB,SGSIN,1,AE
2,BRSSZ009,AEJEA,CNNGB,9,AE
3,BRSSZ009,AEJEA,CNSHA,9,AE
4,BRSSZ009,AEJEA,SGSIN,5,AE
...,...,...,...,...,...
2066,BRSSZ058,ZADUR,GBLGP,1,ZA
2067,BRSSZ058,ZADUR,PTSIE,4,ZA
2068,BRSSZ058,ZAPLZ,PTSIE,8,ZA
2069,BRSSZ058,ZAZBA,ESLPA,1,ZA


In [138]:
df_importacao_agrupado = (
    df_mercante_importacao_agrupado.groupby('codigoTerminalDescarregamento')['conteineres']
      .sum()
      .reset_index()
      .sort_values(by='conteineres', ascending=False)
)
df_importacao_agrupado

,codigoTerminalDescarregamento,conteineres
3,BRSSZ016,159775
6,BRSSZ058,145161
1,BRSSZ009,80644
5,BRSSZ057,1229
4,BRSSZ031,138
2,BRSSZ011,11
0,BRSSZ001,1


In [71]:
sql_fichas_expo = '''SELECT r.nome, count(o.id) as fichas FROM ovr_ovrs o
inner join ovr_recintos r on r.id = o.recinto_id
where o.tipooperacao = 2
and o.datahora between  "2025-06-01" and "2025-10-01"
group by r.nome'''
df_fichas_expo = pd.read_sql(sql_fichas_expo, engine)

In [73]:
nome_para_sigla = {
    'Brasil': 'BTP',
    'SANTOS': 'SBT',
    'EMBRAPORT': 'DPW',
    'ECOPORTO': 'ECO'
}
codigo_para_sigla = {
    'BRSSZ058': 'BTP',
    'BRSSZ016': 'SBT',
    'BRSSZ009': 'DPW',
    'BRSSZ057': 'ECO'
}


df_fichas_expo.nome[1].split()[0]

'ECOPORTO'

In [75]:
df_exportacao_agrupado['sigla'] =  df_exportacao_agrupado.codigoTerminalCarregamento.apply(lambda x: codigo_para_sigla.get(x))
df_exportacao_agrupado_filtrado['sigla'] =  df_exportacao_agrupado_filtrado.codigoTerminalCarregamento.apply(lambda x: codigo_para_sigla.get(x))

In [101]:
df_exportacao_fichas = pd.merge(df_exportacao_agrupado[df_exportacao_agrupado.sigla.notnull()], df_fichas_expo,
                                on='sigla', how='left')[['sigla', 'conteineres', 'fichas']]
df_exportacao_fichas

,sigla,conteineres,fichas
0,BTP,133776,152
1,SBT,95066,1
2,SBT,95066,34
3,DPW,50200,11
4,ECO,2827,14


In [132]:
# Lista das colunas numéricas para calcular percentage
colunas = ['conteineres_total', 'conteineres_risco', 'fichas']

for i, coluna in enumerate(colunas):
    total = df_exportacao_selecao_fichas[coluna].sum()
    percent_col = f'% {i}'   # nome da nova coluna percentual
    df_exportacao_selecao_fichas[percent_col] = (df_exportacao_selecao_fichas[coluna] / total * 100).round(2)
    # Mover a nova coluna para logo após a original
    col_idx = df_exportacao_selecao_fichas.columns.get_loc(coluna)  # índice da coluna original
    col_pct_idx = df_exportacao_selecao_fichas.columns.get_loc(percent_col)  # índice da coluna percentual
    # Rearranjar colunas
    cols = list(df_exportacao_selecao_fichas.columns)
    cols.insert(col_idx + 1, cols.pop(col_pct_idx))  # coloca coluna percentual logo depois da original
    df_exportacao_selecao_fichas = df_exportacao_selecao_fichas[cols]

df_exportacao_selecao_fichas


,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2
0,BTP,133776,47.46,86822,81.99,152,72.04
2,SBT,95066,33.73,11612,10.97,34,16.11
3,DPW,50200,17.81,4932,4.66,11,5.21
4,ECO,2827,1.00,2532,2.39,14,6.64


In [135]:
df_exportacao_selecao_fichas['risco - total'] = df_exportacao_selecao_fichas.conteineres_total - df_exportacao_selecao_fichas.conteineres_risco
df_exportacao_selecao_fichas['risco / total'] = (df_exportacao_selecao_fichas.conteineres_risco / df_exportacao_selecao_fichas.conteineres_total * 100).round(2)
df_exportacao_selecao_fichas

,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2,risco - total,risco / total
0,BTP,133776,47.46,86822,81.99,152,72.04,46954,64.90
1,SBT,95066,33.73,11612,10.97,34,16.11,83454,12.21
2,DPW,50200,17.81,4932,4.66,11,5.21,45268,9.82
3,ECO,2827,1.00,2532,2.39,14,6.64,295,89.56
4,Total,281869,100.00,105898,100.00,211,100.00,175971,37.57


In [146]:
df_impo_expo = pd.merge(df_exportacao_selecao_fichas, df_importacao_agrupado[['sigla', 'conteineres']], on='sigla')
df_impo_expo.rename(columns={'conteineres': 'escaneamento_impo'})
df_impo_expo


,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2,risco - total,risco / total,escaneamento_impo
0,BTP,133776,47.46,86822,81.99,152,72.04,46954,64.90,145161
1,SBT,95066,33.73,11612,10.97,34,16.11,83454,12.21,159775
2,DPW,50200,17.81,4932,4.66,11,5.21,45268,9.82,80644
3,ECO,2827,1.00,2532,2.39,14,6.64,295,89.56,1229


In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204')" # Redex da Localfrio/Movecta e Redex da Santos Brasil Clia
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND numeroConteiner !=''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento

In [10]:
df_entrada_operadores

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento
0,13030920,AHH9B02,ACLU2803211,8931318,2025-10-10 16:47:31,C,E,13431516000132,06882716501,PAULO HENRIQUE FERREIRA DANTAS,31251062356878004885550010000353921413553929,
1,13030922,SJJ0A55,ACLU2794980,8931318,2025-10-10 16:57:58,C,E,13431516000132,05445597628,LUIZ CARLOS DE ARAUJO,31251062356878004885550010000353961413553960,
2,13030930,RFE9G74,GCNU1323485,8931318,2025-10-10 17:09:24,C,E,13431516000132,68547315691,LEONILSON TEODORO PEREIRA,31251062356878004885550010000353951413553955,
3,13030933,RTW5E58,GCNU1365109,8931318,2025-10-10 17:16:49,C,E,03787516000116,12457390850,JOSE ROBERTO ALBINO,31251042135913000246550010000182101957740845,
4,13030935,GXP2J44,HAMU1683170,8931318,2025-10-10 17:18:50,C,E,21425093000176,79017835320,ALAN JORGE SOARES FRAZAO,,
...,...,...,...,...,...,...,...,...,...,...,...,...
23498,13166542,AAW2B18,FCIU7345494,8931404,2025-10-16 23:55:24,C,E,43388833000184,37339188860,RONALDO LUIZ BARROS SOUSA,,
23499,13166543,HVN1C71,MRKU5297030,8931404,2025-10-16 23:55:34,C,E,21425093000176,48302129852,GABRIEL DA SILVA ALVES DE SOUZ,,
23500,13166544,LPA8318,MRSU7477373,8931404,2025-10-16 23:55:51,C,E,49871560000100,37275802830,ISAAC ALVES DA SILVA,35251059179838000218550040000999221497073758,
23501,13166549,GWI8E56,DRYU9083225,8931404,2025-10-16 23:57:41,C,E,05700665000121,06220616820,SERGIO FIRMINO DOS SANTOS,35250955249627000415550010000459501001245483,


In [12]:
df_saida_redex

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,direcao
0,13029438,BYG3F74,FBIU5458946,8931309,2025-10-10 17:02:13,S
1,13029483,AZQ8H09,CGMU5570605,8931309,2025-10-10 17:29:09,S
2,13029487,MIG6A28,TCLU9049458,8931309,2025-10-10 17:41:10,S
3,13029488,CBR3I97,CRXU8685408,8931309,2025-10-10 17:43:54,S
4,13029492,GDG6I86,MSMU3732510,8931309,2025-10-10 18:19:56,S
...,...,...,...,...,...,...
7373,13179523,DPB8C03,MRSU3435364,8933204,2025-10-16 22:43:33,S
7374,13179533,AOQ9F10,GESU6897256,8933204,2025-10-16 22:54:30,S
7375,13179524,HIM3D25,MSDU4395444,8933204,2025-10-16 22:56:01,S
7376,13179526,GIZ4A31,GAOU7730151,8933204,2025-10-16 23:39:08,S


In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'numeroConteiner': 'contêiner',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,id_S_REDEX,Redex,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12189085,GCR3A29,SEGU4965696,8931356,2025-09-08 12:02:31,C,E,05457125000401,89445651553,GIOMAR CARVALHO DA CONCEICAO,35250803816532000190550010002098631350739272,,12211156,8932778,2025-09-08 10:51:00,S,1.191944,None
1,12189109,ESU7C94,CMAU8717266,8931356,2025-09-08 12:10:25,C,E,05457125000401,30142495808,JEFFERSON OLIVEIRA DIAS DOS SANTOS,35250803816532000190550010002098901394579146,,12211157,8932778,2025-09-08 11:06:00,S,1.073611,None
2,12189136,ESU7B97,CMAU8774179,8931356,2025-09-08 12:15:54,C,E,05457125000401,21434097889,LAERCIO ALVES ROCHA,35250803816532000190550010002098611634840526,,12211158,8932778,2025-09-08 11:07:00,S,1.148333,None
3,12189195,FEI3D74,TCNU3567122,8931356,2025-09-08 12:32:31,C,E,05457125000401,36480724801,BRUNO DA SILVA SANTOS,35250903816532000190550010002099771336579381,,12211160,8932778,2025-09-08 11:24:00,S,1.141944,None
4,12189431,ESU7E91,TCNU4651876,8931356,2025-09-08 13:40:07,C,E,05457125000401,02851685848,ANTONIO PEREIRA,35250803816532000190550010002098721417919802,,12211162,8932778,2025-09-08 12:22:00,S,1.301944,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,12296433,SWO3C33,HLXU1061105,8931404,2025-09-11 22:12:22,C,E,51641405000140,28481410934,JORGE BOEIRA DOS SANTOS,35250945736992000158550000000240521547005889,,12262436,8932722,2025-09-10 22:37:54,S,23.574444,None
898,12296598,NTO9D51,HAMU3316213,8931404,2025-09-11 22:58:02,C,E,58130089000786,39914451829,GLEIDSON JUNIOR SANTOS,,,12299824,8932798,2025-09-11 21:27:01,S,1.516944,None
899,12296606,FQN3763,HAMU2437416,8931404,2025-09-11 22:59:08,C,E,58130089000786,37889811802,MIGUEL RUIZ DOS SANTOS,,,12299821,8932798,2025-09-11 20:51:12,S,2.132222,None
900,12296617,ASX1H52,HAMU3055906,8931404,2025-09-11 23:02:20,C,E,58130089000786,00113214588,JORGE CASCIO SOUZA SANTOS SANT,,,12299705,8932798,2025-09-11 21:22:45,S,1.659722,None


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placa', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 3) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placa', 'numeroConteiner'], right_on=['placa', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 3) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) &
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

SyntaxError: invalid syntax (1004912324.py, line 26)